In [1]:
import sys # for automation and parallelisation
manual, scenario = (True, 'base') if 'ipykernel' in sys.argv[0] else (False, sys.argv[1])
if manual:
    %matplotlib inline


Bad key text.latex.preview in file C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
from quetzal.model import stepmodel
from quetzal.io import excel

In [3]:
model_path = '../model/' + scenario + '/'
input_path = '../input/'
input_static_path = '../input_static/'
output_path = '../output/'

In [4]:
# Load scenario parameters
params = excel.read_var(file='../input/parameters.xls', scenario=scenario)

# OD matrix generation

## Needs zone attributes and destination choice model results

## Saves volumes table

In [5]:
# Transport demand segments
segments = [s.strip() for s in params['general']['demand_segments'].split(';')]
purposes = ['commuting', 'business', 'education', 'buy/execute', 'leisure', 'accompany']
purp_n_dict = dict(zip(purposes, [1,2,3,4,6,7]))

In [6]:
# Load zones
sm = stepmodel.read_json(model_path + 'de_zones')
# Sum of trips per day from generation step (million)
sm.zones[[str((s, 'generation')) for s in segments]].sum().sum() / 1e6 / 365

170.00337643647052

## Calculate probabilities of inner/inter-zonal choice

In [7]:
# Load employment data
employment = pd.read_csv(input_static_path + 'spatial_employed_persons_2017_eurostat.csv', encoding='latin-1')
employment['Value'] = (pd.to_numeric(employment['Value'].str.replace(',', '')) * 1000).astype(int)
# Also, load NUTS-names
nuts = gpd.read_file(input_static_path + "spatial_NUTS_RG_01M_2016_4326.geojson")
nuts = nuts[(nuts["CNTR_CODE"]=="DE") & (nuts["LEVL_CODE"]==3)]
# Map differing names of zones to names in employment data
name_dict = {'Burgenlandkreis (DE)': 'Burgenlandkreis',
             'Dillingen an der Donau': 'Dillingen a.d. Donau',
             'Mühldorf am Inn': 'Mühldorf a. Inn',
             'Neumarkt in der Oberpfalz': 'Neumarkt i. d. OPf.',
             'Neustadt an der Aisch-Bad Windsheim': 'Neustadt a. d. Aisch-Bad Windsheim',
             'Neustadt an der Waldnaab': 'Neustadt a. d. Waldnaab',
             'Pfaffenhofen an der Ilm': 'Pfaffenhofen a. d. Ilm',
             'Weiden in der Oberpfalz, Kreisfreie Stadt': 'Weiden i. d. Opf, Kreisfreie Stadt',
             'Wunsiedel im Fichtelgebirge': 'Wunsiedel i. Fichtelgebirge'}
employment['GEO'].replace(name_dict, inplace=True)
zone_dict = employment.set_index('GEO')['Value'].to_dict()
name_dict = nuts.set_index('NUTS_ID')['NUTS_NAME'].to_dict()
sm.zones['employment'] = sm.zones['NUTS_ID'].map(name_dict).map(zone_dict) * sm.zones['pop_share']

In [8]:
# Generate a LoS table
sm.los = sm.zones[['area', 'population', 'employment', 'FID', 'urbanisation']].reset_index(drop=True)
sm.los.rename(columns={'FID': 'origin'}, inplace=True)
# Add one to employment, so that it is 0 nowhere
sm.los['employment'] += 1
# 'route type', which denotes the mode, corresponds to the column of choice
sm.los['route_type'] = 'inner'

In [9]:
# Generate population density
sm.los['pop'] = np.log((1 + sm.los['population']) / sm.los['area'])

In [10]:
# Fill POI values
pois = pd.read_csv(input_path + 'spatial_num_pois_raw.csv', index_col='index')
cats = pd.read_excel(input_path + 'spatial_OSM_POI_list.xlsx', sheet_name='categories')
cats['label'] = (cats['key'] + ' ' + cats['value'].fillna('')).str.strip()
for category, columns in cats.loc[cats['category'].notna()
                                 ].groupby('category').agg(
                                {'label': list})['label'].items():
    sm.los[category] = sm.los['origin'].map(pois[columns].sum(axis=1)).fillna(0)
#sm.los.loc[sm.los['route_type']=='inter', [c for c in cats.loc[cats['category'].notna(), 'category'].unique()]] = 0

In [11]:
# Append rows for inter-zonal choice
sm.los = sm.los.append(pd.DataFrame(data={c: (0 if not c in ['origin'] else sm.los[c])
                                          for c in sm.los.columns}
                                   )).reset_index(drop=True)
sm.los['route_type'] = sm.los['route_type'].replace({0: 'inter'})

In [12]:
# Add accessibility only to inter-zonal rows
# Inner-zonal rows get the mean distance (sqrt of area) multiplied by this segment's relation to CC
car_names = {'no': 0, 'car': 1}
excel = pd.ExcelFile(input_path + 'estimation_results_dest_bounds_inter.xls')
gc_betas = {seg: excel.parse(str(purp_n_dict[seg.split('_')[0]]) + '_' + str(car_names[seg.split('_')[1]])
                            ).set_index('Unnamed: 0').loc['b_gc', 'Value']
            for seg in segments}
cc_inter = pd.read_csv(output_path + scenario + '/mode_choice_od_composite_cost.csv',
                       index_col=['origin', 'destination'])
def logsum(v, param):
    # Choose the same formulation as in cal21
    #return np.log(np.sum(np.exp(v * param)))
    return v.mean()

# Take distance-CC relation from calibration notebook (cal21)
cc_dist_rel = {4: {1: 0.011613001557131047, 0: 0.005701226399690235},
               6: {1: 0.0030793685683175556, 0: 0.0020149513324241156},
               7: {1: 0.014463195082016702, 0: 0.026725078510779394},
               1: {1: 0.03083701740307867, 0: 0.0062200467131824},
               3: {1: 0.007339966675576075, 0: 0.012619421637167493},
               2: {1: 0.0004948050099610248, 0: 0.001182918043521359}}

for seg in segments:
    sm.los[seg+'_accessibility'] = [logsum(cc_inter.xs(o, level=0)[seg], gc_betas[seg])
                                    if choice=='inter'
                                    else np.power(area, 0.5) * cc_dist_rel[int(purp_n_dict[seg.split('_')[0]])
                                                                          ][int(car_names[seg.split('_')[1]])]
                                    for o, area, choice in zip(sm.los['origin'],
                                                               sm.los['area'],
                                                               sm.los['route_type'])]

In [13]:
# Add columns for quetzal functions
sm.los['route_types'] = [(i,) for i in sm.los['route_type']]
# The choice (route_type) will be evalated for every OD pair
sm.los['destination'] = 'DUMMY'

In [14]:
sm.los.sample(3)

,area,population,employment,origin,urbanisation,route_type,pop,childcare,daily_leisure,higher_education,...,education_no_car_accessibility,education_car_accessibility,buy/execute_no_car_accessibility,buy/execute_car_accessibility,leisure_no_car_accessibility,leisure_car_accessibility,accompany_no_car_accessibility,accompany_car_accessibility,route_types,destination
1989,249.35,13555,5799.943231,DEE07_4,3,inner,3.995727,10,59,0,...,0.199271,0.115904,0.090027,0.183379,0.031818,0.048626,0.422011,0.228386,"(inner,)",DUMMY
1511,128.73,66980,29135.967280,DEA1F_5,2,inner,6.254447,12,125,1,...,0.143179,0.083279,0.064686,0.131760,0.022861,0.034938,0.303221,0.164098,"(inner,)",DUMMY
4120,0.00,0,0.000000,DED2F_1,0,inter,0.000000,0,0,0,...,42.499459,10.293109,10.524832,11.971374,13.216522,7.149494,56.801887,11.130708,"(inter,)",DUMMY


In [15]:
# Load estimation results
excel = pd.ExcelFile(input_path + 'estimation_results_go.xls')
car_name = ['_no_car', '_car']
params_est = {seg: excel.parse(str(purp_n_dict[seg.split('_')[0]]) + '_' + str(car_names[seg.split('_')[1]])
                              ).set_index('Unnamed: 0')
              for seg in segments}

In [16]:
# Set nesting structure
nests = pd.DataFrame(index=list(sm.los['route_type'].unique()) + ['root'],
                     data={s: ['root']*len(sm.los['route_type'].unique()) + [np.nan] for s in segments})
nests.index.name = 'route_type'
nests.columns.name = 'segment'
sm.mode_nests = nests
sm.mode_nests

segment,commuting_no_car,commuting_car,business_no_car,business_car,education_no_car,education_car,buy/execute_no_car,buy/execute_car,leisure_no_car,leisure_car,accompany_no_car,accompany_car
route_type,,,,,,,,,,,,
inner,root,root,root,root,root,root,root,root,root,root,root,root
inter,root,root,root,root,root,root,root,root,root,root,root,root
root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Nest parameters
sm.logit_scales = pd.DataFrame(index=list(sm.los['route_type'].unique()),
                               data={seg: 1 for seg in segments})
sm.logit_scales.loc['root'] = [1 for seg in segments]
sm.logit_scales.columns.name = 'segment'
sm.logit_scales.index.name = 'route_type'
sm.logit_scales

segment,commuting_no_car,commuting_car,business_no_car,business_car,education_no_car,education_car,buy/execute_no_car,buy/execute_car,leisure_no_car,leisure_car,accompany_no_car,accompany_car
route_type,,,,,,,,,,,,
inner,1,1,1,1,1,1,1,1,1,1,1,1
inter,1,1,1,1,1,1,1,1,1,1,1,1
root,1,1,1,1,1,1,1,1,1,1,1,1


In [18]:
# Add inner-zonal attraction from size variables
for car in car_name:
    def get_b(p, b):
        return max(0, params_est[p+car].loc[b, 'Value'])

    sm.los['commuting'+car+'_attr'] = np.log(sm.los['employment']).clip(lower=0)
    sm.los['business'+car+'_attr'] = np.log(sm.los['employment']).clip(lower=0)
    sm.los['education'+car+'_attr'] = \
        np.log(1 + sm.los['childcare']# * get_b('education', 'b_edu1') 
               + sm.los['school']# * get_b('education', 'b_edu2')
               + sm.los['higher_education']).clip(lower=0)
    sm.los['buy/execute'+car+'_attr'] = \
        np.log(1 + sm.los['shop']# * get_b('buy/execute', 'b_shop1')
               + sm.los['medical']# * get_b('buy/execute', 'b_shop2')
               + sm.los['special_shop']).clip(lower=0)
    sm.los['leisure'+car+'_attr'] = \
        np.log(1 + sm.los['daily_leisure']# * get_b('leisure', 'b_leisure1')
               + sm.los['holiday']# * get_b('leisure', 'b_leisure2')
               + sm.los['occasional_leisure']).clip(lower=0)
    sm.los['accompany'+car+'_attr'] = \
        np.log(1 + sm.los['medical']# * get_b('accompany', 'b_accom1')
               + sm.los['school']# * get_b('accompany', 'b_accom2')
               + sm.los['childcare']).clip(lower=0)

In [19]:
# Beta parameters for logit function
acc_cols = [seg+'_accessibility' for seg in segments]
attr_cols = [seg+'_attr' for seg in segments]
sm.utility_values = pd.DataFrame(index=acc_cols + attr_cols,
                                 data={seg: 0 for seg in segments})
sm.utility_values.loc['mode_utility'] = [1 for seg in segments]
for a,s in zip(acc_cols, segments):
    sm.utility_values.loc[a, s] = params_est[s].loc['b_gc', 'Value']
for a,s in zip(attr_cols, segments):
    sm.utility_values.loc[a, s] = params_est[s].loc['b_size', 'Value']
#sm.utility_values.loc['pop'] = [params_est[s].loc['b_pop', 'Value'] for s in segments]
#sm.utility_values.loc['urbanisation'] = [params_est[s].loc['b_urban', 'Value'] for s in segments]
sm.utility_values.columns.name = 'segment'
sm.utility_values.index.name = 'value'
sm.utility_values

segment,commuting_no_car,commuting_car,business_no_car,business_car,education_no_car,education_car,buy/execute_no_car,buy/execute_car,leisure_no_car,leisure_car,accompany_no_car,accompany_car
value,,,,,,,,,,,,
commuting_no_car_accessibility,-1.208694,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
commuting_car_accessibility,0.000000,-0.071304,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
business_no_car_accessibility,0.000000,0.000000,-3.049759,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
business_car_accessibility,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
education_no_car_accessibility,0.000000,0.000000,0.000000,0.000000,-0.387783,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
education_car_accessibility,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
buy/execute_no_car_accessibility,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
buy/execute_car_accessibility,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.006494,0.000000,0.000000,0.000000,0.000000
leisure_no_car_accessibility,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [20]:
# Alternative-specific constants
sm.mode_utility = pd.DataFrame(index=list(sm.los['route_type'].unique()),
                               data={s: 0 for s in segments})
#sm.mode_utility.loc['inner'] = [params_est[s].loc['asc_inner', 'Value'] for s in segments]
sm.mode_utility.loc['inter'] = [params_est[s].loc['asc_inter', 'Value'] for s in segments]
sm.mode_utility.columns.name = 'segment'
sm.mode_utility.columns.name = 'route_type'
sm.mode_utility

route_type,commuting_no_car,commuting_car,business_no_car,business_car,education_no_car,education_car,buy/execute_no_car,buy/execute_car,leisure_no_car,leisure_car,accompany_no_car,accompany_car
inner,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
inter,14.994145,6.75201,26.425511,4.544036,13.721747,5.966987,8.233287,5.218174,5.635653,5.078072,8.074906,5.395102


In [21]:
# Run the logit model
sm.segments = segments
# calculate utilities
sm.los.drop([c for c in sm.los.columns if 'probability' in c or 'utility' in c],
            axis=1, inplace=True)
sm.analysis_mode_utility(how='main')
# Run the logit model to calculate probabilities
sm.step_logit(keep_od_tables=False, workers=4, decimals=10, nchunks=12)

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 46.23it/s]


In [22]:
# Get the probabilities of inner-zonal travel by zone
inner_probs = sm.los.loc[sm.los['route_type']=='inner'].set_index('origin')[
    [(s, 'probability') for s in segments]]
inner_probs.columns = segments

In [23]:
# average probabilities of inner-zonal travel
sm.los.loc[sm.los['route_type']=='inner'].groupby('urbanisation').mean()[[(seg, 'probability') for seg in segments]]

,"(commuting_no_car, probability)","(commuting_car, probability)","(business_no_car, probability)","(business_car, probability)","(education_no_car, probability)","(education_car, probability)","(buy/execute_no_car, probability)","(buy/execute_car, probability)","(leisure_no_car, probability)","(leisure_car, probability)","(accompany_no_car, probability)","(accompany_car, probability)"
urbanisation,,,,,,,,,,,,
1,0.999983,0.509037,0.019933,0.517602,0.998868,0.261995,0.380542,0.490439,0.366819,0.414848,0.809953,0.362613
2,0.999068,0.312559,0.005692,0.349312,0.992637,0.073634,0.102590,0.234085,0.168274,0.211004,0.475455,0.128700
3,0.994380,0.209078,0.002693,0.256280,0.971240,0.028497,0.029623,0.111405,0.093680,0.124457,0.211725,0.052830


In [24]:
len(sm.los.loc[sm.los.isna().any(axis=1)])

0

In [25]:
# If all that doesn't make sense:
# Take inner-zonal probabilities from MiD2017 data by urbanisation degree
inner_probs = pd.read_csv(input_path + 'inner-zonal_probabilities.csv', index_col=0)
inner_probs

,commuting_no_car,commuting_car,business_no_car,business_car,education_no_car,education_car,buy/execute_no_car,buy/execute_car,leisure_no_car,leisure_car,accompany_no_car,accompany_car
urbanisation,,,,,,,,,,,,
1,0.819107,0.723101,0.768868,0.660143,0.896435,0.867771,0.905745,0.817660,0.805125,0.730426,0.807273,0.813512
2,0.500000,0.399088,0.682540,0.449192,0.675676,0.616700,0.700272,0.648435,0.617801,0.553961,0.816667,0.635259
3,0.675926,0.474006,0.586957,0.502472,0.607143,0.672573,0.846899,0.703759,0.754777,0.604209,0.840000,0.700693


## Compute volumes (OD matrix)

$F_{ods} = N_{os} * p^{dest}_{ods} * (1 - p^{inner}_{os})$

Indecies:
* o: origin zone
* d: destination zone
* s: demand segment

In [26]:
# Load probability results from logit step
dm = stepmodel.read_zippedpickles(model_path + 'de_destination_choice')

utility_values: 100%|████████████████████████████████████████████████████████████████████| 7/7 [00:06<00:00,  1.12it/s]


In [27]:
# Extract probability values and create new table from it
prob = dm.probabilities.set_index(['origin', 'segment']).drop(columns=['destination', 'root'])
prob.columns.name = 'destination'
volumes = prob.stack('destination').unstack('segment').reset_index()
volumes.columns.name = None
for seg in set(segments) - set(volumes.columns):
    volumes[seg] = 0
volumes.sample(2)

,origin,destination,accompany_car,accompany_no_car,business_car,business_no_car,buy/execute_car,buy/execute_no_car,commuting_car,commuting_no_car,education_car,education_no_car,leisure_car,leisure_no_car
4390345,DEE05_4,DEB3G_1,4.122727e-10,3.251565e-07,8.247120e-07,0.000005,2.092415e-15,1.493429e-09,1.519017e-16,7.611155e-11,6.664133e-20,2.820248e-12,4.957882e-07,0.0
683869,DE21B_1,DE94C_0,5.217817e-10,6.036745e-07,5.669893e-09,0.000001,5.903903e-23,1.237200e-13,9.135265e-25,1.557074e-15,9.134620e-31,5.875373e-18,1.394280e-06,0.0


In [28]:
# Create inner-zonal probabilities
in_mask = volumes['origin']==volumes['destination']
if len(volumes.loc[in_mask]) == 0:
    volumes = volumes.append(pd.DataFrame({'origin': list(sm.zones.index),
                                           'destination': list(sm.zones.index)})
                            ).reset_index()
    in_mask = volumes['origin']==volumes['destination']
urban_dict = sm.zones['urbanisation'].to_dict()
for seg in segments:
    probs = inner_probs[seg]
    # Correct probabilities in volumes
    if len(probs) < len(sm.zones): # aggregated probabilities by urbanisation degree
        volumes.loc[in_mask, seg] = volumes.loc[in_mask, 'origin'].map(urban_dict).map(probs)
        volumes.loc[~in_mask, seg] = volumes.loc[~in_mask, seg] * \
                                     volumes.loc[~in_mask, 'origin'].map(urban_dict).map(1 - probs)
    else:
        volumes.loc[in_mask, seg] = volumes.loc[in_mask, 'origin'].map(probs)
        volumes.loc[~in_mask, seg] = volumes.loc[~in_mask, seg] * \
                                     volumes.loc[~in_mask, 'origin'].map(1 - probs)

In [29]:
# make sure all probabilities sum up to 1 for each zone in each segments
#volumes.groupby('origin')[segments].sum().describe()

In [30]:
# calculate number of trips
for s in segments:
    generation_dict = sm.zones[str((s, 'generation'))].to_dict()
    volumes[s] = volumes[s] * volumes['origin'].map(generation_dict)

In [31]:
# check the sum (million trips per day)
volumes[segments].sum().sum() / 1e6 / 365

158.9628398297865

## Sparsify the OD set

Reduce the number of OD pairs to a sample while keeping the number of trips constant.

In [33]:
# Only sample, if the parameter is specified
sample_size = params['general']['od_sparse_sample']
if sample_size > 0:
    # Set a stable seed, generating the same OD set in every scenario run
    np.random.seed(42)
    # Choice probabilities of OD pairs weighted by trip volumes
    od_probabilities = volumes[segments].sum(axis=1) / volumes[segments].sum().sum()
    sample = np.random.choice(a=volumes.index,
                              size=sample_size,
                              p=od_probabilities)
    # Reduce the volumes matrix
    for seg in segments:
        expansion = volumes[seg].sum() / sample_size
        volumes[seg] = pd.Series(sample).value_counts() * expansion

In [34]:
# Check the sum (million trips per day)
volumes.loc[volumes.notna().all(axis=1), segments].sum().sum() / 365 / 1e6

158.96283982978647

In [35]:
volumes = volumes.loc[volumes.notna().all(axis=1)]

## Save

In [36]:
# Save volumes table
sm.volumes = volumes.reset_index(drop=True)
sm.to_zippedpickles(model_path + 'de_volumes', only_attributes=['volumes'])

volumes: 100%|█████████████████████████████████████████████████████████████████████████| 38/38 [00:01<00:00, 23.43it/s]
